In [9]:
import os
import math

import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### save path
savePath = r'F:\JJM\\'

#params

behavCamFrameRate=15
miniscopeCamFrameRate=20
downSampleFactorForVelocity = '.2S'
medianFilter = '1S'
sessionPath = 'F:\JJM\GRIN018_H16_M13_S53\concactenated_behavCam01_behavCam41_LocationOutput.csv'

print(sessionPath.split(os.sep)[:-1])
# load eZ track output and behavior camera timestamps from miniscope software 
ezTrackOutput = pd.read_csv(sessionPath)
timestampfile = pd.read_table('\\'.join(sessionPath.split(os.sep)[:-1])+'\\timestamp.dat')

behavCamTimeStamps = timestampfile[timestampfile['camNum'] == 1]
miniscopeCamTimeStamps = timestampfile[timestampfile['camNum'] == 0]
behavCamTimeStamps['frameNum'][0:len(ezTrackOutput)]
miniscopeCamTimeStamps['frameNum'][0:len(ezTrackOutput)]

#trim both to length of whichever is shorter 
L = min(len(miniscopeCamTimeStamps), len(behavCamTimeStamps)); miniscopeCamTimeStamps, behavCamTimeStamps = miniscopeCamTimeStamps.iloc[:L], behavCamTimeStamps.iloc[:L]

# convert to formats from "_v4" sheet 
timestampfile = (
    behavCamTimeStamps
        .rename(columns={
            "frameNum": "Frame Number",
            "sysClock": "Time Stamp (ms)"
        })
        .drop(columns=["camNum", "buffer"])
        .reset_index(drop=True)
)

miniscope_timestampfile = (
    miniscopeCamTimeStamps
        .rename(columns={
            "frameNum": "Frame Number",
            "sysClock": "Time Stamp (ms)"
        })
        .drop(columns=["camNum", "buffer"])
        .reset_index(drop=True)
)

    
timestampfile_td = timestampfile.set_index(pd.to_timedelta(np.linspace(0, (len(timestampfile)-1)*(1/behavCamFrameRate), len(timestampfile)), unit='s'), drop=False)
  
miniscopetimestamp_td_originalHz = miniscope_timestampfile.set_index(pd.to_timedelta(np.linspace(0, (len(miniscope_timestampfile)-1)*(1/miniscopeCamFrameRate), len(miniscope_timestampfile)), unit='s'), drop=False)

behavCam_frames = []
sys_clock_behavCam = []
#create "key" for aligning miniscope frames to timestamp file
#then create behavior TD and align
for msCam_frame in tqdm(range(0, len(miniscopetimestamp_td_originalHz['Frame Number']))):
    #get sys clock time of each miniscope recorded frame
    #sys_clock_msCam = time_stamps['sysClock'].loc[msCam_frame]
    #find behav cam frame closest to sys clock time of ms frame
    behavCam_frame = list(timestampfile_td.iloc[(timestampfile_td['Time Stamp (ms)']-miniscopetimestamp_td_originalHz['Time Stamp (ms)'].iloc[msCam_frame]).abs().argsort()[:1]].index)[0]
    #this is the behavCamIndex that is closest to the corresponding miniscope frame 
    behavCam_frames.append(behavCam_frame)
    sys_clock_behavCam.append(timestampfile_td.loc[behavCam_frame]['Time Stamp (ms)'])

behavCamIdxToAlign = [timestampfile_td.index.get_loc(idx) for idx in behavCam_frames]
#ezTrackOutput
miniscopetimestamp_td_originalHz['closestBehavCamFrameIdx'] = behavCamIdxToAlign

X_coor=[]
Y_coor=[]
Distance_px=[] 
Distance_cm=[]

for i in tqdm(miniscopetimestamp_td_originalHz['closestBehavCamFrameIdx'].values):
    X_coor.append(ezTrackOutput.loc[miniscopetimestamp_td_originalHz['closestBehavCamFrameIdx'].values[i]]['X'])
    Y_coor.append(ezTrackOutput.loc[miniscopetimestamp_td_originalHz['closestBehavCamFrameIdx'].values[i]]['Y'])
    Distance_px.append(ezTrackOutput.loc[miniscopetimestamp_td_originalHz['closestBehavCamFrameIdx'].values[i]]['Distance_px'])
    Distance_cm.append(ezTrackOutput.loc[miniscopetimestamp_td_originalHz['closestBehavCamFrameIdx'].values[i]]['Distance_cm'])
    
miniscopetimestamp_td_originalHz['X_coor'] = X_coor
miniscopetimestamp_td_originalHz['Y_coor'] = Y_coor
miniscopetimestamp_td_originalHz['Distance_px'] = Distance_px
miniscopetimestamp_td_originalHz['Distance_cm'] = Distance_cm

# calculate cmPerPixel
cmPerPixel = ezTrackOutput['Distance_cm'][1]/ezTrackOutput['Distance_px'][1]

print(cmPerPixel)
def euclideanDistance(row):
    return(math.sqrt(row['X_coor']**2+row['Y_coor']**2))  
def euclideanDistancePixels(row, cmPerPixel):
    return(math.sqrt(row['X_coor']**2+row['Y_coor']**2)*cmPerPixel)
def calcVelocity(row, cmPerPixel, samplingRate):
    #euclidean distance divided by sampling rate, multiplied by cm/pixel
    return((math.sqrt(row['X_coor']**2+row['Y_coor']**2)/samplingRate)*cmPerPixel)
    
# 
samplingRateOrig = 0.05
miniscopetimestamp_td_originalHz['Velocity'] = miniscopetimestamp_td_originalHz.diff().apply(lambda row : calcVelocity(row, cmPerPixel, samplingRateOrig), axis=1)
miniscopetimestamp_td_originalHz['Distance'] = miniscopetimestamp_td_originalHz.diff().apply(lambda row : euclideanDistancePixels(row, cmPerPixel), axis=1)

## in doing the downsampling, break apart X_coor, Y_coor columns to to .mean() instead of .bfill()

# downsample to .1S
miniscopeTimeStampsDownsampled_1 = miniscopetimestamp_td_originalHz.resample('.1S').bfill()
X_Y_Downsampled_1 = miniscopetimestamp_td_originalHz[['X_coor', 'Y_coor']].resample('.1S').mean()
miniscopeTimeStampsDownsampled_1[['X_coor', 'Y_coor']] = X_Y_Downsampled_1

# downsample to .2S
miniscopeTimeStampsDownsampled_2 = miniscopetimestamp_td_originalHz.resample('.2S').bfill()
X_Y_Downsampled_2 = miniscopetimestamp_td_originalHz[['X_coor', 'Y_coor']].resample('.2S').mean()
miniscopeTimeStampsDownsampled_2[['X_coor', 'Y_coor']] = X_Y_Downsampled_2

#
samplingRate_1 = 0.1
miniscopeTimeStampsDownsampled_1['Velocity'] = miniscopeTimeStampsDownsampled_1.diff().apply(lambda row : calcVelocity(row, cmPerPixel, samplingRate_1), axis=1)
miniscopeTimeStampsDownsampled_1['Distance'] = miniscopeTimeStampsDownsampled_1.diff().apply(lambda row : euclideanDistancePixels(row, cmPerPixel), axis=1)

#
samplingRate_2 = 0.2
miniscopeTimeStampsDownsampled_2['Velocity'] = miniscopeTimeStampsDownsampled_2.diff().apply(lambda row : calcVelocity(row, cmPerPixel, samplingRate_2), axis=1)
miniscopeTimeStampsDownsampled_2['Distance'] = miniscopeTimeStampsDownsampled_2.diff().apply(lambda row : euclideanDistancePixels(row, cmPerPixel), axis=1)

# save these for comparison
miniscopetimestamp_td_originalHz.to_csv(savePath+os.path.split(sessionPath)[0].split('\\')[-1]+'velocity_origHz.csv')
miniscopeTimeStampsDownsampled_1.to_csv(savePath+os.path.split(sessionPath)[0].split('\\')[-1]+'velocity_resampled_1.csv')
miniscopeTimeStampsDownsampled_2.to_csv(savePath+os.path.split(sessionPath)[0].split('\\')[-1]+'velocity_resampled_2.csv')

# save indicies where mouse is in given velocity bins 

velocityBins = [(0, 0.5), (0.5, 1), (1, 2), (2, 4), (4, 8), (8, math.inf)]
v_bin_dict = {}

#save data frames of bin indicies in dictionary
for bin_idx in range(len(velocityBins)):
    v_bin_dict[bin_idx]=pd.DataFrame([miniscopeTimeStampsDownsampled_2.index.get_loc(miniscopeTimeStampsDownsampled_2[(miniscopeTimeStampsDownsampled_2['Velocity']>velocityBins[bin_idx][0]) & (miniscopeTimeStampsDownsampled_2['Velocity']<velocityBins[bin_idx][1])].index[i]) 
    for i in range(len(miniscopeTimeStampsDownsampled_2[(miniscopeTimeStampsDownsampled_2['Velocity']>velocityBins[bin_idx][0]) & (miniscopeTimeStampsDownsampled_2['Velocity']<velocityBins[bin_idx][1])].index))], columns=['IndiciesInVBin'])
#save to csv files
for bin_idx in range(len(velocityBins)):
    v_bin_dict[bin_idx].to_csv(savePath+os.path.split(sessionPath)[0].split('\\')[-1]+'_velocityBin'+str(bin_idx)+'.csv')


C:\Users\scanimage\AppData\Local\Temp\ipykernel_22172\1051854094.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


['F:', 'JJM', 'GRIN018_H16_M13_S53']


100%|██████████████████████████████████████████████| 40052/40052 [00:15<00:00, 2640.16it/s]


0.11181533560963901
